In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
from tqdm.notebook import tqdm 
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam

In [ ]:
!unzip -qq /content/drive/'My Drive'/Code/tomatoes.zip

In [ ]:
dataset_path = '/content/PlantVillage'

class_name = []
images = []
labels = []
for index, folder in enumerate(os.listdir(dataset_path)):
  class_name.append(folder)
  image_path = os.path.join(dataset_path, folder)
  # print(image_path, len(os.listdir(image_path)))
  for image in os.listdir(image_path):
    labels.append(index)
    images.append(os.path.join(dataset_path, folder, image))

X = []
for index, path in enumerate(tqdm(images)):
  try:
    X.append(cv2.resize(cv2.imread(path),dsize=(256,256)))
  except:
    labels.pop(index)
X = np.array(X)
y = np.array(labels)
Y = to_categorical(y, num_classes=len(class_name))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,stratify = Y, shuffle=True, random_state=100)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test, shuffle=True, random_state=100)
print(X_train.shape, X_test.shape,X_val.shape)
print(y_train.shape, y_test.shape,y_val.shape)

(9902, 256, 256, 3) (1238, 256, 256, 3) (1238, 256, 256, 3)
(9902, 5) (1238, 5) (1238, 5)


In [ ]:
def get_model():
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    input = Input(shape=(256, 256, 3))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = model_vgg16_conv(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(5, activation='softmax', name='predictions')(x)
    # multi output
    output = Dense(5,activation = 'softmax', name='root')(x)

    # Compile
    model = Model(input, x)
    optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()

    return model

vggmodel = get_model()
filepath="/content/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

vgghist=vggmodel.fit(aug.flow(X_train, y_train, batch_size=64),
                               epochs=20, steps_per_epoch=len(X_train)//64,
                               validation_data=aug.flow(X_val,y_val,
                               batch_size=64), callbacks=callbacks_list)

vggmodel.save("vggmodell1.h5")

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 3)       84        
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)              

In [ ]:
from keras.models import load_model
#model = load_model('/content/weights-04-0.93.hdf5')
#print("[INFO] Calculating model accuracy")
#scores = model.evaluate(X_test, y_test)
#scores = vggmodell.evaluate(X_test, y_test)
#print(f"Test Accuracy: {scores[1]*100}")
for i in range(1,20)
  modellink = '/content/weights-' + '0' + '%d' + '.hdf5' %(i)
  model = load_model(modellink)
  scores = model.evaluate(X_test, y_test)
  print(f"Test Accuracy: {scores[1]*100}")

SyntaxError: ignored